In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 5 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Challenge : convergence du gradient conjugué
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#    

# Challenge : convergence du gradient conjugué

In [ ]:
import numpy as np
from scipy.sparse import diags

def eig_val_1d(nx, dx, i):
    a = (4/(dx**2))
    return a*np.sin((np.pi*i)/(2*(nx+1)))**2

def eig_vec(nx, dx, i):
    j = np.arange(1,nx+1)
    return np.sqrt(2/(nx+1)) * np.sin(j*i*np.pi/(nx+1))

def conjugate_gradient(a, b, eps=1.e-6):
    xk = np.zeros(b.size)
    norm_b = np.linalg.norm(b)

    rk = b - a.dot(xk)
    pk = rk
    rkm1 = rk
    
    hist_norm_rk = []

    for k in range(b.size):
        apk = a.dot(pk)
        alpha = np.dot(rk,rk) / np.dot(pk, apk)
        xk = xk + alpha*pk
        rk = rk - alpha*apk
        norm_rk = np.linalg.norm(rk)
        hist_norm_rk.append(norm_rk/norm_b)
        if norm_rk/norm_b < eps: break
        beta = np.dot(rk,rk) / np.dot(rkm1, rkm1)
        pk = rk + beta*pk
        rkm1 = rk

    print(f"  Nombre d'itérations = {k+1}")
    print(f"  ||A.xk - b|| / ||b|| = {norm_rk/norm_b}")

    return xk, hist_norm_rk

## Equation de Poisson

On souhaite résoudre le problème elliptique constitué par l'équation de Poisson dans le cas 1D soumise à des conditions aux limites de type de Dirichlet homogène :

$$
\left\{
\begin{aligned}
- & u''(x)  =  f(x) \quad \text{dans} \; \Omega = [0,1] \quad \text{avec} \; f(x)=1\\
  & u(0)  =  0 \; \text{et} \; u(1) = 0
\end{aligned}
\right.
$$

La discrétisation par différence finie permet de se ramener à la résolution d'un système linéaire $Ax=b$.

On résoud ce système avec l'algorithme du gradient conjugué en choisissant l'itéré initial $x_0 = 0$ pour différent second membre.

In [ ]:
nx = 100
dx = 1/(nx+1)

# construction de la matrice creuse
diag = np.repeat(2/dx**2, nx)
diag_x = np.repeat(-1/dx**2, nx-1)
a  = diags([diag, diag_x, diag_x], [0, -1, 1])

**Cas d'un second membre $b$ toutes les composantes de valent 1**

In [ ]:
b = np.ones(nx)
print(f"Cas 1d : nx = {nx}")
print(f"Taille de la matrice : ({nx} x {nx})")
u, _ = conjugate_gradient(a, b)

**Cas d'un second membre $b$ égale à un vecteur propre de $A$**

In [ ]:
b = eig_vec(nx, dx, 3)

print(f"Cas 1d : nx = {nx}")
print(f"Taille de la matrice : ({nx} x {nx})")

x, _ = conjugate_gradient(a, b)

Comment expliquer que l'algorithme converge en 1 itération lorsque le second membre est un vecteur propre de $A$ ? 